In [5]:
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import torch
import pandas as pd

# Load the dataset
lang = 'en'
data = load_dataset(f"Cohere/wikipedia-22-12", lang)

model_name = "BAAI/bge-base-en"  
model = SentenceTransformer(model_name)



Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/352 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
def generate_embeddings_batch(text_batch):
    embeddings = model.encode(text_batch,batch_size = 512,convert_to_numpy =True,device ="cuda")
    return embeddings

In [ ]:
import numpy as np
batch_size = 60000

def generate_embeddings_batch(text_batch):
    embeddings = model.encode(text_batch,batch_size = 512,convert_to_numpy =True,device ="cuda")
    return embeddings


for i in range(0, len(data['train']), batch_size):
    batch = data['train'][i:i+batch_size]
    title_batch = batch["title"]
    text_batch = batch["text"]
    
    combined_batch = [f"{title} {text}" for title, text in zip(title_batch, text_batch)]
    embeddings_batch = generate_embeddings_batch(combined_batch)
    
    print("Sentence Shappe:", embeddings_batch.shape)
    print(type(embeddings_batch))

    np.save(f"newBGE/batch_{i // batch_size:05d}.npy", embeddings_batch) 


In [23]:
from autofaiss import build_index

build_index(embeddings="newBGE", index_path="index_folderV2/knn.index",
            index_infos_path="index_folderV2/index_infos.json", max_index_memory_usage="11G",
            metric_type = 'l2',
            index_key	="OPQ64_192,IVF65536_HNSW32,PQ64",
            # index_key	="Flat",
            # make_direct_map=True,
            index_param = "nprobe",
            max_index_query_time_ms = 50,
            use_gpu = True,
            current_memory_available="40G")

0